In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

df = pd.read_csv("heart.csv")

df['target'] = df['num'].apply(lambda x: 1 if x > 0 else 0)
df = df[['age', 'chol', 'target']]

X = df[['age', 'chol']].values
y = df['target'].values

y = np.where(y == 0, -1, 1)

scaler = StandardScaler()
X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [2]:
print(df)

     age   chol  target
0     63  233.0       0
1     67  286.0       1
2     67  229.0       1
3     37  250.0       0
4     41  204.0       0
..   ...    ...     ...
915   54  333.0       1
916   62  139.0       0
917   55  223.0       1
918   58  385.0       0
919   62  254.0       1

[920 rows x 3 columns]


In [6]:
class CustomSVM:
    def __init__(self, lr=0.001, lambda_param=0.01, n_iters=1000):
        self.lr = lr
        self.lambda_param = lambda_param
        self.n_iters = n_iters
        self.w = None
        self.b = None

    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.w = np.zeros(n_features)
        self.b = 0

        for _ in range(self.n_iters):
            for idx, x_i in enumerate(X):
                condition = y[idx] * (np.dot(x_i, self.w) + self.b) >= 1
                if condition:
                    self.w -= self.lr * (2 * self.lambda_param * self.w)
                else:
                    self.w -= self.lr * (2 * self.lambda_param * self.w - np.dot(x_i, y[idx]))
                    self.b -= self.lr * y[idx]
        
            self.w = np.clip(self.w, -10, 10)
            self.b = np.clip(self.b, -10, 10)


    def predict(self, X):
        approx = np.dot(X, self.w) + self.b
        preds = np.sign(approx)
        preds[np.isnan(preds)] = 1
        return preds



In [7]:
svm = CustomSVM()
svm.fit(X_train, y_train)
predictions = svm.predict(X_test)

from sklearn.metrics import accuracy_score, classification_report

print("🔹 Accuracy:", accuracy_score(y_test, predictions))
print("\n🔹 Classification Report:\n", classification_report(y_test, predictions))


🔹 Accuracy: 0.592391304347826

🔹 Classification Report:
               precision    recall  f1-score   support

          -1       0.00      0.00      0.00        75
           1       0.59      1.00      0.74       109

    accuracy                           0.59       184
   macro avg       0.30      0.50      0.37       184
weighted avg       0.35      0.59      0.44       184



C:\Users\medha\OneDrive\Desktop\dl-lab\myenv\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\medha\OneDrive\Desktop\dl-lab\myenv\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\medha\OneDrive\Desktop\dl-lab\myenv\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capi